
Connect to Google Drive for datasets (colab)

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install dependencies (colab)

In [ ]:
#!pip install simpleitk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.5 MB/s eta 0:00:00


Dependencies

In [2]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import scipy

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Resampling MRI image to specified voxel spacing <br>
https://gist.github.com/mrajchl/ccbd5ed12eb68e0c1afc5da116af614a


In [3]:
def resample_img(itk_image, out_spacing=[2.0, 0.6, 0.6], is_label=False):

    # Resample images to 2 0.6 0.6 spacing with SimpleITK
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkLinear)

    return resample.Execute(itk_image)

Dataset Directories <Br>
Comment out directory not in use


In [3]:
#Local Directories Toy Dataset
dummy_train_img_dir = pathlib.Path(r"D:\Spider Data/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"D:\Spider Data/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"D:\Spider Data/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"D:\Spider Data/dummy_test_labels")

'''
#Colab Google Drive Directories Toy Dataset
dummy_train_img_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_test_labels")
'''


'\n#Colab Google Drive Directories Toy Dataset\ndummy_train_img_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_train_images")\ndummy_train_label_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_train_labels")\ndummy_test_img_dir = pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_test_images")\ndummy_test_label_dir= pathlib.Path(r"/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_test_labels")\n'

Image class

In [4]:
class Mri:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case

        #resampling
        #mri_mha_resampled = resample_img(mri_mha)
        #TODO separate resample (bilinear, nearestNeighbor) for images and labels

        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array

        #transpose array to format z x y
        if(mri_a.shape[0] > mri_a.shape[1] or mri_a.shape[0] > mri_a.shape[2]): #if z axis isn't first
          mri_a = np.transpose(mri_a, (2, 0, 1))

        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Check for sorted directory and dimension order <Br>
Get max dimension on x y for zero padding 

In [5]:
label_dir_list = os.listdir(dummy_train_label_dir)
image_dir_list = os.listdir(dummy_train_img_dir)

image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)

dim1_list = []
dim2_list = []

dirlen = len(os.listdir(dummy_train_label_dir))

for idx in range(0, dirlen):
  img_path = dummy_train_img_dir.joinpath(image_dir_list[idx])
  label_path = dummy_train_label_dir.joinpath(label_dir_list[idx])

  image = Mri(img_path)
  label = Mri(label_path)
  '''
  print(idx, "image: ", image.hu_a.shape)
  print(idx, "label: ", label.hu_a.shape)
  '''
  dim1_list.append(image.hu_a.shape[1])
  dim2_list.append(image.hu_a.shape[2])


print(dim1_list)

x_dim_max = max(dim1_list)
y_dim_max = max(dim2_list)

print("x max:", max(dim1_list))
print("y max:", max(dim2_list))

x_dim_max = 912
y_dim_max = 912



[578, 578, 294, 346, 389, 486, 553, 553, 512, 512, 512, 512, 384, 384, 464, 610, 896, 768, 590, 589, 320, 384, 748, 748, 899, 899, 448, 448, 495, 495, 512, 512, 351, 351, 512, 512, 323, 323, 344, 344, 462, 462, 600, 600, 402, 402]
x max: 899
y max: 896


Zero Padding to max res of dataset  

In [6]:
def pad_to_resolution(input_tensor, target_resolution):
    current_resolution = input_tensor.shape[-2:]

    if current_resolution == target_resolution:
        return input_tensor  # No padding needed

    pad_height = target_resolution[0] - current_resolution[0]
    pad_width = target_resolution[1] - current_resolution[1]

    # Calculate pad values
    top_pad = pad_height // 2
    bottom_pad = pad_height - top_pad
    left_pad = pad_width // 2
    right_pad = pad_width - left_pad

    # Apply padding
    padded_tensor = torch.nn.functional.pad(input_tensor, (left_pad, right_pad, top_pad, bottom_pad), mode='constant', value=0)

    return padded_tensor

Dataset Class

In [7]:
class SpiderDataset(Dataset):
    def __init__(self, labels_dir, img_dir, transform=None, target_transform=None):
        self.labels_dir = labels_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.labels_dir))

    def __getitem__(self, idx):
        label_dir_list = os.listdir(self.labels_dir)
        image_dir_list = os.listdir(self.img_dir)

        image_dir_list = natsorted(image_dir_list)
        label_dir_list = natsorted(label_dir_list)

        img_path = self.img_dir.joinpath(image_dir_list[idx])
        label_path = self.labels_dir.joinpath(label_dir_list[idx])

        image = Mri(img_path)
        label = Mri(label_path)

        #image = self.transform(image)
        #label = self.target_transform(label)

        #comment out the part not being used whether for 3d or 2d model 
    
        '''
        #3d tensor for 3D CNN
        image_tensor = torch.from_numpy(image.hu_a)
        label_tensor = torch.from_numpy(label.hu_a)
        '''

        #2d tensor for 2D CNN, get random slice from image 
        rand_idx = np.random.randint(0, image.hu_a.shape[0])

        image_tensor = torch.from_numpy(image.hu_a[rand_idx])
        label_tensor = torch.from_numpy(label.hu_a[rand_idx])
        
        image_tensor = image_tensor.to(torch.float32)
        label_tensor = label_tensor.to(torch.float32)
        
        image_tensor = pad_to_resolution(image_tensor, [x_dim_max, y_dim_max])
        label_tensor = pad_to_resolution(label_tensor, [x_dim_max, y_dim_max])

        image_tensor = image_tensor.unsqueeze(0)
        label_tensor = label_tensor.unsqueeze(0)

        return image_tensor, label_tensor

#toy train test dataset to test network running
dummy_train_set = SpiderDataset(dummy_train_label_dir, dummy_train_img_dir)
dummy_test_set = SpiderDataset(dummy_test_label_dir, dummy_test_img_dir)

print("train dataset len",dummy_train_set.__len__())
print("test dataset len",dummy_test_set.__len__())


train dataset len 46
test dataset len 10


Testing on single image

In [ ]:
dir_glob= dummy_train_img_dir.glob("*.mha")

for idx in dir_glob:
  image = Mri(path = "/content/gdrive/MyDrive/Spider Dummy Dataset/dummy_test_images/30_t1.mha")
  print(image.hu_a)
  break

Unet LabML

In [8]:
"""
---
title: U-Net
summary: >
    PyTorch implementation and tutorial of U-Net model.
---

# U-Net

This is an implementation of the U-Net model from the paper,
[U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597).

U-Net consists of a contracting path and an expansive path.
The contracting path is a series of convolutional layers and pooling layers,
where the resolution of the feature map gets progressively reduced.
Expansive path is a series of up-sampling layers and convolutional layers
where the resolution of the feature map gets progressively increased.

At every step in the expansive path the corresponding feature map from the contracting path
concatenated with the current feature map.

![U-Net diagram from paper](unet.png)

Here is the [training code](experiment.html) for an experiment that trains a U-Net
on [Carvana dataset](carvana.html).
"""
import torch
import torchvision.transforms.functional
from torch import nn


class DoubleConvolution(nn.Module):
    """
    ### Two $3 \times 3$ Convolution Layers

    Each step in the contraction path and expansive path have two $3 \times 3$
    convolutional layers followed by ReLU activations.

    In the U-Net paper they used $0$ padding,
    but we use $1$ padding so that final feature map is not cropped.
    """

    def __init__(self, in_channels: int, out_channels: int):
        """
        :param in_channels: is the number of input channels
        :param out_channels: is the number of output channels
        """
        super().__init__()

        # First $3 \times 3$ convolutional layer
        self.first = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        # Second $3 \times 3$ convolutional layer
        self.second = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()

    def forward(self, x: torch.Tensor):
        # Apply the two convolution layers and activations
        x = self.first(x)
        x = self.act1(x)
        x = self.second(x)
        return self.act2(x)


class DownSample(nn.Module):
    """
    ### Down-sample

    Each step in the contracting path down-samples the feature map with
    a $2 \times 2$ max pooling layer.
    """

    def __init__(self):
        super().__init__()
        # Max pooling layer
        self.pool = nn.MaxPool2d(2)

    def forward(self, x: torch.Tensor):
        return self.pool(x)


class UpSample(nn.Module):
    """
    ### Up-sample

    Each step in the expansive path up-samples the feature map with
    a $2 \times 2$ up-convolution.
    """
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        # Up-convolution
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x: torch.Tensor):
        return self.up(x)


class CropAndConcat(nn.Module):
    """
    ### Crop and Concatenate the feature map

    At every step in the expansive path the corresponding feature map from the contracting path
    concatenated with the current feature map.
    """
    def forward(self, x: torch.Tensor, contracting_x: torch.Tensor):
        """
        :param x: current feature map in the expansive path
        :param contracting_x: corresponding feature map from the contracting path
        """

        # Crop the feature map from the contracting path to the size of the current feature map
        contracting_x = torchvision.transforms.functional.center_crop(contracting_x, [x.shape[2], x.shape[3]])
        # Concatenate the feature maps
        x = torch.cat([x, contracting_x], dim=1)
        #
        return x


class UNet(nn.Module):
    """
    ## U-Net
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        :param in_channels: number of channels in the input image
        :param out_channels: number of channels in the result feature map
        """
        super().__init__()

        # Double convolution layers for the contracting path.
        # The number of features gets doubled at each step starting from $64$.
        self.down_conv = nn.ModuleList([DoubleConvolution(i, o) for i, o in
                                        [(in_channels, 64), (64, 128), (128, 256), (256, 512)]])
        # Down sampling layers for the contracting path
        self.down_sample = nn.ModuleList([DownSample() for _ in range(4)])

        # The two convolution layers at the lowest resolution (the bottom of the U).
        self.middle_conv = DoubleConvolution(512, 1024)

        # Up sampling layers for the expansive path.
        # The number of features is halved with up-sampling.
        self.up_sample = nn.ModuleList([UpSample(i, o) for i, o in
                                        [(1024, 512), (512, 256), (256, 128), (128, 64)]])
        # Double convolution layers for the expansive path.
        # Their input is the concatenation of the current feature map and the feature map from the
        # contracting path. Therefore, the number of input features is double the number of features
        # from up-sampling.
        self.up_conv = nn.ModuleList([DoubleConvolution(i, o) for i, o in
                                      [(1024, 512), (512, 256), (256, 128), (128, 64)]])
        # Crop and concatenate layers for the expansive path.
        self.concat = nn.ModuleList([CropAndConcat() for _ in range(4)])
        # Final $1 \times 1$ convolution layer to produce the output
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x: torch.Tensor):
        """
        :param x: input image
        """
        # To collect the outputs of contracting path for later concatenation with the expansive path.
        pass_through = []
        # Contracting path
        for i in range(len(self.down_conv)):
            # Two $3 \times 3$ convolutional layers
            x = self.down_conv[i](x)
            # Collect the output
            pass_through.append(x)
            # Down-sample
            x = self.down_sample[i](x)

        # Two $3 \times 3$ convolutional layers at the bottom of the U-Net
        x = self.middle_conv(x)

        # Expansive path
        for i in range(len(self.up_conv)):
            # Up-sample
            x = self.up_sample[i](x)
            # Concatenate the output of the contracting path
            x = self.concat[i](x, pass_through.pop())
            # Two $3 \times 3$ convolutional layers
            x = self.up_conv[i](x)

        # Final $1 \times 1$ convolution layer
        x = self.final_conv(x)

        #
        return x

Create Unet Instance







In [9]:
input_channels = 1 #Hounsfield scale
output_channels = 3 #Vertebra, disc and spinal canal masks
model = UNet(in_channels = input_channels, out_channels = output_channels)
model.to(device)

UNet(
  (down_conv): ModuleList(
    (0): DoubleConvolution(
      (first): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act1): ReLU()
      (second): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act2): ReLU()
    )
    (1): DoubleConvolution(
      (first): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act1): ReLU()
      (second): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act2): ReLU()
    )
    (2): DoubleConvolution(
      (first): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act1): ReLU()
      (second): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act2): ReLU()
    )
    (3): DoubleConvolution(
      (first): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act1): ReLU()
      (second): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act2): ReLU()
    )
 

Hyperparameters


In [14]:
epochs = 1 #testing
lr = 0.001 #testing
batchsize = 2 #testing
loss_func = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

Dataloader

In [11]:
dummy_train_dataloader = DataLoader(dummy_train_set, batch_size = batchsize, shuffle=True)
dummy_test_dataloader = DataLoader(dummy_test_set, batch_size = batchsize, shuffle=True)

Dataloader Iterate through Z Axis of tensor (3D tensor)

In [63]:
for images, masks in dummy_test_dataloader:
  for i in images, masks:
    print(i.shape)
    break


torch.Size([10, 899, 896])


One Epoch <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [12]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dummy_train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)

        print(inputs.shape)

        #inputs = inputs.transpose(-1,0)
        #labels = labels.transpose(-1,0)
        #inputs = inputs.reshape(inputs.shape(1), inputs.shape(0), inputs.shape(3), inputs.shape(4))
        #labels = labels.reshape(labels.shape(1), labels.shape(0), labels.shape(3), labels.shape(4))

      
        # Zero your gradients for every batch!
        optim.zero_grad()

        #2d
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optim.step()
        
        '''
        #run everything while indexing through z axis
        for axis in images, masks:
          for idx in range(0, axis.size(1)):
            # Zero your gradients for every batch!
            optim.zero_grad()
            print(inputs[:, idx, : ,:])
            # Make predictions for this batch
            outputs = model(inputs[:, idx, : ,:])

            # Compute the loss and its gradients
            loss = loss_func(outputs[:, idx, : ,:], labels[:, idx, : ,:])
            loss.backward()

            # Adjust learning weights
            optim.step()
        '''
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(dummy_train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

Train Loop <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html


In [13]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/spider_seg_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 1

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(dummy_test_dataloader):
            vinputs, vlabels = vdata

            #from tutorial code start
            voutputs = model(vinputs)
            vloss = loss_func(voutputs, vlabels)
            running_vloss += vloss
            #from tutorial code end

            for slices in vdata.size(1):
              voutputs = model(vinputs)
              vloss = loss_func(voutputs, vlabels)
              running_vloss += vloss


    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
torch.Size([2, 1, 912, 912])


OutOfMemoryError: CUDA out of memory. Tried to allocate 408.00 MiB (GPU 0; 6.00 GiB total capacity; 5.20 GiB already allocated; 0 bytes free; 5.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF